In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

#from matplotlib import style
#Seaborn's visual styling was inspired by ggplot, change matplotlib plot style to match
#style.use('ggplot')

# Data Fields

** About the Data **  

** Data Fields **


# Import the Data

In [3]:
#open and read the file into a pandas data frame
projFile = "Projects.csv"
schedFile = "Schedules.csv"
budFile = "Budgets.csv"

In [4]:
#view the header names for files
list(pd.read_csv(projFile, nrows=1).columns), list(pd.read_csv(schedFile, nrows=1).columns), list(pd.read_csv(budFile, nrows=1).columns)

(['ProjectId',
  'FMSID',
  'Benchmark (Y/N)',
  'DivisionName',
  'UnitName',
  'ProjectDescription',
  'Scope',
  'WebScope',
  'Borough',
  'ClientAgencyCode',
  'Sponsor',
  'PhaseName',
  'ProjectType',
  'Priority',
  'DesignContractType',
  'ConstructionContractType',
  'SourceSystem',
  'MultipleFMSIds',
  'BenchmarkStatus',
  'DesignFiscalYear',
  'ConstructionFiscalYear',
  'CreatedDate',
  'ManageAgency'],
 ['ProjectId',
  'Benchmark (Y/N)',
  'ProjectedProjectStart',
  'ProjectedDesignStart',
  'ProjectedDesignCompletion',
  'ProjectedConstructionStart',
  'ProjectedConstructionCompletion',
  'OrgProjectStart',
  'OrgDesignStart',
  'OrgDesignCompletion',
  'OrgConstructionStart',
  'OrgConstructionCompletion',
  'ClientExtProjectStart',
  'ClientExtOrgDesignStart',
  'ClientExtOrgDesignCompletion',
  'ClientExtOrgConstructionStart',
  'ClientExtOrgConstructionCompletion',
  'ActProjectStart',
  'ActDesignStart',
  'ActDesignCompletion',
  'ActConstructionStart',
  'ActCons

In [5]:
#set the columns to read in for each file
projCols = ([
  'ProjectId',
  'FMSID',
  #'Benchmark (Y/N)',
  'DivisionName',
  'UnitName',
  #'ProjectDescription',
  #'Scope',
  #'WebScope',
  'Borough',
  #'ClientAgencyCode',
  'Sponsor',
  'PhaseName',
  'ProjectType',
  'Priority',
  'DesignContractType',
  'ConstructionContractType',
  'SourceSystem',
  'MultipleFMSIds',
  #'BenchmarkStatus',
  'DesignFiscalYear',
  'ConstructionFiscalYear',
  #'CreatedDate'
  #'ManageAgency'
  ])

schedCols =([
  'ProjectId',
  #'Benchmark (Y/N)',
  #'ProjectedProjectStart',
  #'ProjectedDesignStart',
  #'ProjectedDesignCompletion',
  #'ProjectedConstructionStart',
  #'ProjectedConstructionCompletion',
  #'OrgProjectStart',
  #'OrgDesignStart',
  #'OrgDesignCompletion',
  #'OrgConstructionStart',
  #'OrgConstructionCompletion',
  #'ClientExtProjectStart',
  #'ClientExtOrgDesignStart',
  #'ClientExtOrgDesignCompletion',
  #'ClientExtOrgConstructionStart',
  #'ClientExtOrgConstructionCompletion',
  'ActProjectStart',
  'ActDesignStart',
  'ActDesignCompletion',
  'ActConstructionStart',
  'ActConstructionCompletion',
  #'ClientExtBaseDesignCompletion',
  #'ClientExtBaseConstructionCompletion',
  #'DesignBaselineProjectStart',
  #'ConstructionBaselineProjectStart',
  #'DesignBaselineDesignStart',
  #'ConstructionBaselineDesignStart',
  #'OrgSchematicDesignCompletion',
  #'ProjectedSchematicDesignCompletion',
  'ActSchematicDesignCompletion',
  #'DesignBaselineSchematicDesignCompletion',
  #'ConstructionBaselineSchematicDesignCompletion',
  #'OrgDesignDevelopmentCompletion',
  #'ProjectedDesignDevelopmentCompletion',
  'ActDesignDevelopmentCompletion',
  #'DesignBaselineDesignDevelopmentCompletion',
  #'ConstructionBaselineDesignDevelopmentCompletion',
  #'DesignBaselineDesignCompletion',
  #'ConstructionBaselineDesignCompletion',
  #'DesignBaselineConstructionStart',
  #'ConstructionBaselineConstructionStart',
  #'DesignBaselineConstructionCompletion',
  #'ConstructionBaselineConstructionCompletion',
  #'OrgProjectCloseout',
  #'ProjectedProjectCloseout',
  'ActProjectCloseout'
  #,
  #'DesignBaselineProjectCloseout',
  #'ConstructionBaselineProjectCloseout',
  #'CreatedDate',
  #'ModifiedDate'
  ])

budCols = ([
  #'MANAGING_AGCY_CD',
  #'SHORT_DESCR',
  'ClntAgcyCode',
  'AgcyName',
  'project_id',
  'OCCURRENCE_ID',
  'pobject',
  #'phasetype',
  #'PlannedAmt',
  'CommittedAmt'
  #,
  #'LiquidatedAmt',
  #'AvailableAmt'
  #'DataAsOF'
  ])

Dates cannot be parsed when the file is read in because they contain blank values, they have to be read in as strings and then converted after they are read in.

In [6]:
#Set up datatypes for each file, use "category" for strings where 
#possible, more memory efficient than object (str)

proj_dtypes = {
  'ProjectId': 'str',
  'FMSID': 'str',
  #'Benchmark (Y/N)': 'category',
  'DivisionName': 'category',
  'UnitName': 'category',
  #'ProjectDescription': ,
  #'Scope':,
  #'WebScope':,
  'Borough': 'category',
  #'ClientAgencyCode':,
  'Sponsor': 'category',
  'PhaseName': 'category',
  'ProjectType': 'category',
  'Priority':'category',
  'DesignContractType':'category',
  'ConstructionContractType':'category',
  'SourceSystem':'category',
  'MultipleFMSIds':'category',
  #'BenchmarkStatus':,
  'DesignFiscalYear': 'str', #blank values so can't be int
  'ConstructionFiscalYear':'str' #blank values so can't be int
  #'CreatedDate':'str'
  #'ManageAgency': 'category'
    }


In [7]:
#all except the first two fields are dates that need to be read in as
#str and then converted to date fields
sched_dtypes = {}
#add the first two fields
sched_dtypes.update({'ProjectId': 'str'})
#for the remaining fields if it has 'Act' in the field name add it
#to the dictionary as a 'str'
for field in schedCols[2:]:
    if 'Act' in field:
        sched_dtypes[field] ='str'
#sched_dtypes

In [8]:
bud_dtypes = {
  #'MANAGING_AGCY_CD',
  #'SHORT_DESCR',
  'ClntAgcyCode':'category',
  'AgcyName':'category',
  'project_id':'str',  #ties to FMSID in projects
  'OCCURRENCE_ID':'category',
  'pobject':'category',
  #'phasetype':'category',
  #'PlannedAmt': 'float',
  'CommittedAmt': 'float'  #has blank values cannot be int
  #,
  #'LiquidatedAmt':'float',
  #'AvailableAmt':'float',
  #'DataAsOF'
  }

In [9]:
#read in the projects file
df_proj = pd.read_csv(projFile, usecols = projCols, 
                 dtype = proj_dtypes)

In [10]:
print(len(df_proj))
df_proj.head()

1000


,ProjectId,FMSID,DivisionName,UnitName,Borough,Sponsor,PhaseName,ProjectType,Priority,DesignContractType,ConstructionContractType,SourceSystem,MultipleFMSIds,DesignFiscalYear,ConstructionFiscalYear
0,ACEDOS501,ACEDOS501,Public Buildings,ONENYC,Unknown,DCAS,Completed Project,Upgrade,Standard,Requirements TO,DDC Managed,PIMS,FALSE,NaN,NaN
1,ACEDOS503,ACEDOS503,Public Buildings,TRANSPORTATION,Queens,Sanitation,Construction 0-25%,New Construction,Court Mandated,CM-Build,CM-Build,PIMS,FALSE,2017,2017
2,ACEDOT601,ACEDOT601,Public Buildings,TRANSPORTATION,Brooklyn,DCAS,NaN,Upgrade,Critical,NaN,NaN,PIMS,FALSE,NaN,NaN
3,ACEDOT602,ACEDOT602,Public Buildings,TRANSPORTATION,Manhattan,DCAS,NaN,Upgrade,Critical,NaN,NaN,PIMS,FALSE,NaN,NaN
4,ACSLSPCAR,ACSLSPCAR,Public Buildings,CORRECTIONS,Brooklyn,Children Services,Substantial Completion,Renovation,Emergency,Standard Consultant,CM-Managed,PIMS,FALSE,NaN,NaN


In [11]:
#read in the schedule file
df_sched = pd.read_csv(schedFile, usecols = schedCols, 
                 dtype = sched_dtypes)

In [12]:
print(len(df_sched))
df_sched.head()

1000


,ProjectId,ActProjectStart,ActDesignStart,ActDesignCompletion,ActConstructionStart,ActConstructionCompletion,ActSchematicDesignCompletion,ActDesignDevelopmentCompletion,ActProjectCloseout
0,ACEDOS501,2014-07-11 00:00:00.0000000 +00:00,2014-07-18 00:00:00.0000000 +00:00,2014-12-23 00:00:00.0000000 +00:00,2015-08-11 00:00:00.0000000 +00:00,2016-04-08 18:00:00.0000000 +00:00,2014-09-01 00:00:00.0000000 +00:00,2014-10-01 00:00:00.0000000 +00:00,2017-02-07 00:00:00.0000000 +00:00
1,ACEDOS503,2017-02-13 00:00:00.0000000 +00:00,2017-02-14 17:00:00.0000000 +00:00,2017-05-24 00:00:00.0000000 +00:00,2017-07-03 17:00:00.0000000 +00:00,NaN,2017-03-14 00:00:00.0000000 +00:00,2017-04-14 00:00:00.0000000 +00:00,NaN
2,ACEDOT601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACEDOT602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACSLSPCAR,2015-08-20 00:00:00.0000000 +00:00,2015-10-15 00:00:00.0000000 +00:00,NaN,2016-02-17 00:00:00.0000000 +00:00,2017-03-23 00:00:00.0000000 +00:00,NaN,NaN,NaN


In [13]:
#read in the schedule file
df_bud = pd.read_csv(budFile, usecols = budCols, 
                 dtype = bud_dtypes)

In [14]:
print(len(df_bud))
df_bud.head()

1000


,ClntAgcyCode,AgcyName,project_id,OCCURRENCE_ID,pobject,CommittedAmt
0,856,DEPARTMENT OF CITYWIDE ADMINISTRATIVE SERVICES,ACEDOS501,1,CONS,1164680.0
1,856,DEPARTMENT OF CITYWIDE ADMINISTRATIVE SERVICES,ACEDOS501,100,IFDS,129999.0
2,96,HUMAN RESOURCES ADMINISTRATION,ACSBELL,3,DSGN,70000.0
3,96,HUMAN RESOURCES ADMINISTRATION,ACSBELL,36,CONS,0.0
4,96,HUMAN RESOURCES ADMINISTRATION,ACSBELL,63,EQFN,0.0


# Data Cleaning 

Exploration (for all fields – for ID fields is it unique, what are the possible values, frequency by value, what is the datatype, are there missing values)

Check for errors, deal with special values, convert data into different formats, and perform calculations  

In [15]:
#create a function to view field details
def field_details(df):
    for col in list(df):
        null_values = sum(df[col].isnull())
        count = df[col].value_counts()
        values = df[col].unique()
        values_list = list(values)
        if len(values_list) > 25:
            values_list = str(len(values_list)) + ' category values'
        print('{}: max count = {}, null values = {}, category values = {}\n'.format(
            col,max(count),null_values,values_list))

In [16]:
#view projects tab details
field_details(df_proj)

ProjectId: max count = 5, null values = 0, category values = 945 category values

FMSID: max count = 7, null values = 4, category values = 939 category values

DivisionName: max count = 475, null values = 6, category values = ['Public Buildings', nan, 'Infrastructure', 'Safety and Site Support', 'Agency-wide', 'Law']

UnitName: max count = 139, null values = 4, category values = 38 category values

Borough: max count = 188, null values = 317, category values = ['Unknown', 'Queens', 'Brooklyn', 'Manhattan', 'Citywide', nan, 'Bronx', 'Staten Is.']

Sponsor: max count = 325, null values = 73, category values = ['DCAS', 'Sanitation', 'Children Services', nan, 'Aging', 'Environmental Protection', 'Education', 'Corrections', 'Health', 'Trans. & Env. Protection', 'Police', 'Unknown', 'Fire', 'Homeless Services', 'Transportation', 'HPD', 'Parks & Recreation', 'SocialServices-HRA', 'Emergency Management', 'Libraries-BPL', 'Libraries-NYPL', 'Libraries-QBPL']

PhaseName: max count = 115, null val

all fields except ProjectId have missing values  
there are multiple observations with the same ProjectId and/or FMSID   
  
Review duplicates

In [17]:
list(df_proj)

['ProjectId',
 'FMSID',
 'DivisionName',
 'UnitName',
 'Borough',
 'Sponsor',
 'PhaseName',
 'ProjectType',
 'Priority',
 'DesignContractType',
 'ConstructionContractType',
 'SourceSystem',
 'MultipleFMSIds',
 'DesignFiscalYear',
 'ConstructionFiscalYear']

In [18]:
df_proj[df_proj.duplicated(subset='ProjectId', keep=False)]

,ProjectId,FMSID,DivisionName,UnitName,Borough,Sponsor,PhaseName,ProjectType,Priority,DesignContractType,ConstructionContractType,SourceSystem,MultipleFMSIds,DesignFiscalYear,ConstructionFiscalYear
34,CC253B-1,CC253B,Public Buildings,Human Services,NaN,DCAS,Acceptance/Final Completion,Renovation,Standard,Requirements TO,Requirements TO,PIMS,TRUE,NaN,NaN
35,CC253B-1,CC253B-1,Public Buildings,Human Services,NaN,DCAS,Acceptance/Final Completion,Renovation,Standard,Requirements TO,Requirements TO,PIMS,TRUE,NaN,NaN
96,DOIT1CQN,DOIT1CNET,Public Buildings,HUMAN SERVICES,Queens,DCAS,Maintenance,Upgrade,Standard,Standard Consultant,Requirements TO,PIMS,TRUE,NaN,NaN
97,DOIT1CQN,DOIT1CQN,Public Buildings,HUMAN SERVICES,Queens,DCAS,Maintenance,Upgrade,Standard,Standard Consultant,Requirements TO,PIMS,TRUE,NaN,NaN
116,E12-0043A,E12-0043,Public Buildings,ONENYC,Unknown,DCAS,Construction 0-25%,Upgrade,Standard,Requirements TO,DDC Managed,PIMS,TRUE,NaN,NaN
117,E12-0043A,E12-0043A,Public Buildings,ONENYC,Unknown,DCAS,Construction 0-25%,Upgrade,Standard,Requirements TO,DDC Managed,PIMS,TRUE,NaN,NaN
151,PW311T01C,PW311S01C,Safety and Site Support,Site Engineering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,PW311T01C,PW311T01C,Safety and Site Support,Site Engineering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,GE354,GE354,Infrastructure,MANHATTAN & CITYWIDE,Citywide,Environmental Protection,Completed Project,Water,Emergency,In-House,DEP Managed,PIMS,TRUE,2013,2014
181,GE354,GE-354,Infrastructure,MANHATTAN & CITYWIDE,Citywide,Environmental Protection,Completed Project,Water,Emergency,In-House,DEP Managed,PIMS,TRUE,2013,2014


In [19]:
df_proj[df_proj.duplicated(subset=['FMSID'], keep=False)]

,ProjectId,FMSID,DivisionName,UnitName,Borough,Sponsor,PhaseName,ProjectType,Priority,DesignContractType,ConstructionContractType,SourceSystem,MultipleFMSIds,DesignFiscalYear,ConstructionFiscalYear
13,E3000M019R,E3000DDC,Public Buildings,Schools,NaN,Education,Completed Project,Renovation,Mayoral In,NaN,CM-Build,NaN,NaN,NaN,NaN
14,E3000M020,E3000DDC,Public Buildings,Schools,Manhattan,Education,Completed Project,Renovation,Mayoral In,NaN,CM-Build,NaN,NaN,NaN,1997
15,E3000M020A,E3000DDC,Public Buildings,Schools,NaN,Education,Completed Project,Renovation,Mayoral In,NaN,CM-Manage,NaN,NaN,NaN,NaN
22,C75FCADGRVC,C75-FCAD,Public Buildings,Corrections,NaN,Corrections,Substantial Completion,Renovation,Standard,Standard Consultant,DDC Managed,Project Info,FALSE,NaN,NaN
23,E3000M180P,E3000DDC,Public Buildings,Schools,NaN,Education,Completed Project,Renovation,Mayoral In,NaN,CM-Build,NaN,NaN,NaN,NaN
24,E3000M180R,E3000DDC,Public Buildings,Schools,NaN,Education,Completed Project,Renovation,Mayoral In,NaN,CM-Build,NaN,NaN,NaN,NaN
25,C114NEWRI,C114NEWRI,Public Buildings,CORRECTIONS,Bronx,Corrections,Cancelled Project,New Construction,Standard,Standard Consultant,NaN,PIMS,FALSE,NaN,NaN
26,C114NEWRIP1,C114NEWRI,Public Buildings,Corrections,NaN,Corrections,Substantial Completion,New Construction,Standard,Standard Consultant,JOCs,Project Info,FALSE,NaN,NaN
27,C114NEWRIP2,C114NEWRI,Public Buildings,CORRECTIONS,Bronx,Corrections,Construction 75%-Subst.,New Construction,Standard,Standard Consultant,CM-Managed,PIMS,FALSE,NaN,2015
29,C75-FCADEMT,C75-FCAD,Public Buildings,Corrections,NaN,Corrections,Construction 75%-Subst.,Renovation,Standard,Standard Consultant,JOCs,PIMS,FALSE,NaN,NaN


** Neither ProjectID nor FMSID is unique, but there should be a master table where they are with master information for the project, another timestamped table can contain the changes **

In [20]:
#view schedules tab details
field_details(df_sched)

ProjectId: max count = 1, null values = 0, category values = 1000 category values

ActProjectStart: max count = 19, null values = 78, category values = 673 category values

ActDesignStart: max count = 10, null values = 173, category values = 644 category values

ActDesignCompletion: max count = 7, null values = 490, category values = 392 category values

ActConstructionStart: max count = 12, null values = 515, category values = 398 category values

ActConstructionCompletion: max count = 5, null values = 717, category values = 246 category values

ActSchematicDesignCompletion: max count = 8, null values = 476, category values = 400 category values

ActDesignDevelopmentCompletion: max count = 6, null values = 825, category values = 148 category values

ActProjectCloseout: max count = 1, null values = 977, category values = ['2017-02-07 00:00:00.0000000 +00:00', nan, '2016-11-14 00:00:00.0000000 +00:00', '2015-12-05 00:00:00.0000000 +00:00', '2017-05-25 00:00:00.0000000 +00:00', '2015-07-

In [21]:
#change date fields to date types in schedule tab
#Convert all nan observations to 1/1/1900 so they can be easily 
#identified and excluded

df_sched.fillna(value= '1900-01-01',inplace=True)

In [22]:
df_sched.head()

,ProjectId,ActProjectStart,ActDesignStart,ActDesignCompletion,ActConstructionStart,ActConstructionCompletion,ActSchematicDesignCompletion,ActDesignDevelopmentCompletion,ActProjectCloseout
0,ACEDOS501,2014-07-11 00:00:00.0000000 +00:00,2014-07-18 00:00:00.0000000 +00:00,2014-12-23 00:00:00.0000000 +00:00,2015-08-11 00:00:00.0000000 +00:00,2016-04-08 18:00:00.0000000 +00:00,2014-09-01 00:00:00.0000000 +00:00,2014-10-01 00:00:00.0000000 +00:00,2017-02-07 00:00:00.0000000 +00:00
1,ACEDOS503,2017-02-13 00:00:00.0000000 +00:00,2017-02-14 17:00:00.0000000 +00:00,2017-05-24 00:00:00.0000000 +00:00,2017-07-03 17:00:00.0000000 +00:00,1900-01-01,2017-03-14 00:00:00.0000000 +00:00,2017-04-14 00:00:00.0000000 +00:00,1900-01-01
2,ACEDOT601,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
3,ACEDOT602,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
4,ACSLSPCAR,2015-08-20 00:00:00.0000000 +00:00,2015-10-15 00:00:00.0000000 +00:00,1900-01-01,2016-02-17 00:00:00.0000000 +00:00,2017-03-23 00:00:00.0000000 +00:00,1900-01-01,1900-01-01,1900-01-01


In [23]:
#truncate format to Year-month-day
df_sched.iloc[:,1:] = df_sched.iloc[:,1:].apply(lambda x: x.str[:10])

In [24]:
df_sched.head()

,ProjectId,ActProjectStart,ActDesignStart,ActDesignCompletion,ActConstructionStart,ActConstructionCompletion,ActSchematicDesignCompletion,ActDesignDevelopmentCompletion,ActProjectCloseout
0,ACEDOS501,2014-07-11,2014-07-18,2014-12-23,2015-08-11,2016-04-08,2014-09-01,2014-10-01,2017-02-07
1,ACEDOS503,2017-02-13,2017-02-14,2017-05-24,2017-07-03,1900-01-01,2017-03-14,2017-04-14,1900-01-01
2,ACEDOT601,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
3,ACEDOT602,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
4,ACSLSPCAR,2015-08-20,2015-10-15,1900-01-01,2016-02-17,2017-03-23,1900-01-01,1900-01-01,1900-01-01


In [25]:
#convert to date format
df_sched2 = df_sched.iloc[:,1:].apply(pd.to_datetime,format='%Y-%m-%d')

In [26]:
df_sched2.dtypes

ActProjectStart                   datetime64[ns]
ActDesignStart                    datetime64[ns]
ActDesignCompletion               datetime64[ns]
ActConstructionStart              datetime64[ns]
ActConstructionCompletion         datetime64[ns]
ActSchematicDesignCompletion      datetime64[ns]
ActDesignDevelopmentCompletion    datetime64[ns]
ActProjectCloseout                datetime64[ns]
dtype: object

In [27]:
df_sched2.insert(loc=0, column='ProjectId', value=df_sched['ProjectId'])

In [28]:
df_sched = df_sched2
print(df_sched.dtypes)
df_sched.head()

ProjectId                                 object
ActProjectStart                   datetime64[ns]
ActDesignStart                    datetime64[ns]
ActDesignCompletion               datetime64[ns]
ActConstructionStart              datetime64[ns]
ActConstructionCompletion         datetime64[ns]
ActSchematicDesignCompletion      datetime64[ns]
ActDesignDevelopmentCompletion    datetime64[ns]
ActProjectCloseout                datetime64[ns]
dtype: object


,ProjectId,ActProjectStart,ActDesignStart,ActDesignCompletion,ActConstructionStart,ActConstructionCompletion,ActSchematicDesignCompletion,ActDesignDevelopmentCompletion,ActProjectCloseout
0,ACEDOS501,2014-07-11,2014-07-18,2014-12-23,2015-08-11,2016-04-08,2014-09-01,2014-10-01,2017-02-07
1,ACEDOS503,2017-02-13,2017-02-14,2017-05-24,2017-07-03,1900-01-01,2017-03-14,2017-04-14,1900-01-01
2,ACEDOT601,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
3,ACEDOT602,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
4,ACSLSPCAR,2015-08-20,2015-10-15,1900-01-01,2016-02-17,2017-03-23,1900-01-01,1900-01-01,1900-01-01


In [29]:
field_details(df_sched)

ProjectId: max count = 1, null values = 0, category values = 1000 category values

ActProjectStart: max count = 78, null values = 0, category values = 617 category values

ActDesignStart: max count = 173, null values = 0, category values = 586 category values

ActDesignCompletion: max count = 490, null values = 0, category values = 390 category values

ActConstructionStart: max count = 515, null values = 0, category values = 369 category values

ActConstructionCompletion: max count = 717, null values = 0, category values = 237 category values

ActSchematicDesignCompletion: max count = 476, null values = 0, category values = 400 category values

ActDesignDevelopmentCompletion: max count = 825, null values = 0, category values = 148 category values

ActProjectCloseout: max count = 977, null values = 0, category values = [numpy.datetime64('2017-02-07T00:00:00.000000000'), numpy.datetime64('1900-01-01T00:00:00.000000000'), numpy.datetime64('2016-11-14T00:00:00.000000000'), numpy.datetime64

** There are no duplicate projectIDs in schedule table so this will be used as the master table and information will be brought in from other tables, making assumptions in terms of what to bring in **

In [30]:
#view budgets tab details
field_details(df_bud)

ClntAgcyCode: max count = 343, null values = 0, category values = ['856', '96', '125', '826', '827', '126', '130', '72', '806', '68', '850', '816', '312']

AgcyName: max count = 343, null values = 0, category values = ['DEPARTMENT OF CITYWIDE ADMINISTRATIVE SERVICES', 'HUMAN RESOURCES ADMINISTRATION', 'DEPARTMENT FOR THE AGING', 'DEPT OF ENVIRONMENTAL PROTECTION', 'DEPARTMENT OF SANITATION', 'DEPARTMENT OF CULTURAL AFFAIRS', 'DEPARTMENT OF JUVENILE JUSTICE', 'DEPARTMENT OF CORRECTION', 'HOUSING PRESERVATION AND DEVELOPEMENT', "ADMIN. FOR CHILDREN'S SERVICES", 'DEPT. OF DESIGN & CONSTRUCTION', 'DEPARTMENT OF HEALTH', 'CONFLICTS OF INTEREST BOARD']

project_id: max count = 30, null values = 0, category values = 214 category values

OCCURRENCE_ID: max count = 74, null values = 0, category values = 119 category values

pobject: max count = 428, null values = 0, category values = ['CONS', 'IFDS', 'DSGN', 'EQFN', 'IFSP', 'CTGY', 'EQVH', 'SVCS', 'CNSP', 'OTHR', 'AART', 'IFOT', 'SITE']

Commit

# Create Calculated Fields

** Calculate phase durations for schedules **  
-- check for 1900 dates and skip calc for those

** Milestones	Milestone Range	Phase**  
Project Start	PS to DS	Initiation  
Design Start	DS to DC	Design  
Design Completion	DC to CS	Procurement  
Construction Start	CS to CC	Construction  
Construction Complesion	CC to PC	Project Closeout

In [31]:
#check for dates where the year is 1900 - there's are converted NA values
#if the dates are both valid subtract date 1 from date 2
def date_diff(date1,date2):
    diff = np.where((date1.dt.year != 1900) & (date2.dt.year != 1900), 
             (date2 - date1).dt.days, 
              np.nan)
    return diff

In [32]:
df_sched['Initiation'] = date_diff(df_sched['ActProjectStart'],df_sched['ActDesignStart'])
df_sched['Design'] = date_diff(df_sched['ActDesignStart'],df_sched['ActDesignCompletion'])
df_sched['Procurement'] =  date_diff(df_sched['ActDesignCompletion'],df_sched['ActConstructionStart'])
df_sched['Construction'] = date_diff(df_sched['ActConstructionStart'], df_sched['ActConstructionCompletion'])
df_sched['Project Closeout'] = date_diff(df_sched['ActConstructionCompletion'], df_sched['ActProjectCloseout'])

In [33]:
df_sched.head()

,ProjectId,ActProjectStart,ActDesignStart,ActDesignCompletion,ActConstructionStart,ActConstructionCompletion,ActSchematicDesignCompletion,ActDesignDevelopmentCompletion,ActProjectCloseout,Initiation,Design,Procurement,Construction,Project Closeout
0,ACEDOS501,2014-07-11,2014-07-18,2014-12-23,2015-08-11,2016-04-08,2014-09-01,2014-10-01,2017-02-07,7.0,158.0,231.0,241.0,305.0
1,ACEDOS503,2017-02-13,2017-02-14,2017-05-24,2017-07-03,1900-01-01,2017-03-14,2017-04-14,1900-01-01,1.0,99.0,40.0,NaN,NaN
2,ACEDOT601,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,NaN,NaN,NaN,NaN,NaN
3,ACEDOT602,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,NaN,NaN,NaN,NaN,NaN
4,ACSLSPCAR,2015-08-20,2015-10-15,1900-01-01,2016-02-17,2017-03-23,1900-01-01,1900-01-01,1900-01-01,56.0,NaN,NaN,400.0,NaN


In [34]:
df_sched[['Initiation','Design','Procurement','Construction', 'Project Closeout']].describe()

,Initiation,Design,Procurement,Construction,Project Closeout
count,827.000000,509.000000,407.000000,281.000000,23.000000
mean,393.573156,623.646365,401.120393,613.249110,543.869565
std,517.711116,651.299023,233.589400,488.259718,374.228091
min,-1461.000000,0.000000,0.000000,14.000000,0.000000
25%,87.000000,157.000000,270.500000,319.000000,335.000000
50%,277.000000,459.000000,369.000000,468.000000,434.000000
75%,513.500000,863.000000,501.500000,783.000000,813.500000
max,4416.000000,5402.000000,2849.000000,4385.000000,1133.000000


** Review negative values **

In [35]:
df_sched[df_sched.Initiation < 0 ]

,ProjectId,ActProjectStart,ActDesignStart,ActDesignCompletion,ActConstructionStart,ActConstructionCompletion,ActSchematicDesignCompletion,ActDesignDevelopmentCompletion,ActProjectCloseout,Initiation,Design,Procurement,Construction,Project Closeout
770,LNXEA08BA,2015-11-05,2011-11-05,2017-07-10,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,-1461.0,2074.0,NaN,NaN,NaN


There is only one negative value, and the actual design start is earlier than the project start. Since there is only one observation with this issue it will be dropped from the dataset

In [36]:
#remove the row that contains a negative value for initiation days
df_sched = df_sched[~(df_sched.Initiation < 0)].copy()

In [37]:
len(df_sched)

999

** Using the schedules tab as the primary tab, check how many projectIDs appears in the projects and budget tabs **

In [38]:
#check if projectId in schedules is also in projects and budgets
schedId_in_proj = [x for x in df_sched.ProjectId if x in list(df_proj.ProjectId)]
schedId_in_bud = [x for x in df_proj.FMSID if x in list(df_bud.project_id)]

In [39]:
len(df_sched), len(schedId_in_proj), len(schedId_in_bud)

(999, 857, 20)

** There is a lot of overlap between schedules and projects but not between projects and budgets and therefore not between budgets and schedules **

** Focus on just schedules and projects to begin with **

In [40]:
#number of unique projectIDs in projects
len(df_proj.ProjectId.unique())

945

In [41]:
#try to drop duplicates on the projects dataframe
df_proj_agg = df_proj[['ProjectId','DivisionName','UnitName','Borough','Sponsor','PhaseName',
 'ProjectType', 'Priority', 'DesignContractType', 'ConstructionContractType',
 'SourceSystem', 'MultipleFMSIds', 'DesignFiscalYear','ConstructionFiscalYear']].drop_duplicates()

In [42]:
len(df_proj_agg)

945

** Removed duplicates from projects for the purpose of this analysis, but in a complete analysis we would need to understand why there are duplicates in this tab**   
  
** Next Step: connect the schedule tab to the projects tab by project ID **

In [43]:
df_SP = pd.merge(df_sched, df_proj_agg, on='ProjectId', how='inner')

In [44]:
#check number of observations in the joined dataframe
len(df_SP)

857

In [45]:
#view fields
list(df_SP)

['ProjectId',
 'ActProjectStart',
 'ActDesignStart',
 'ActDesignCompletion',
 'ActConstructionStart',
 'ActConstructionCompletion',
 'ActSchematicDesignCompletion',
 'ActDesignDevelopmentCompletion',
 'ActProjectCloseout',
 'Initiation',
 'Design',
 'Procurement',
 'Construction',
 'Project Closeout',
 'DivisionName',
 'UnitName',
 'Borough',
 'Sponsor',
 'PhaseName',
 'ProjectType',
 'Priority',
 'DesignContractType',
 'ConstructionContractType',
 'SourceSystem',
 'MultipleFMSIds',
 'DesignFiscalYear',
 'ConstructionFiscalYear']

** Consider available fields and what information they can provide **

 'ProjectId' - now unique for this dataset  
  
 ** *Number of Days fields that we now care about* **  
 'Initiation'
 'Design',
 'Procurement',
 'Construction',
 'Project Closeout',
  
 'DivisionName': ['Public Buildings', 'Infrastructure']  
 
 'UnitName': [ONENYC, TRANSPORTATION, CORRECTIONS, EXECUTIVE, Human Services, ..., Transportation, Manhattan & Citywide, Miscellaneous, Libraries, LIBRARIES]
 
 'Borough': 
 
 'Sponsor': [DCAS, Sanitation, Children Services, NaN, Aging, ..., Trans. & Env. Protection, Emergency Management, Libraries-BPL, Libraries-NYPL, Libraries-QBPL]

 'ProjectType': [Upgrade, New Construction, Renovation, Other, Water, ..., Feasibility Study, Street Reconstruction, Street Resurfacing, Street, Ped Ramps]

 'Priority': [Standard, Court Mandated, Critical, Emergency, NaN, Mayoral Initiative, Mayoral In]

 'DesignContractType': [Requirements TO, CM-Build, NaN, Standard Consultant, In-House, ..., JOCS, CM, DDC Managed, DDC Managed Consultant, Pass Through]
 
 'ConstructionContractType': [DDC Managed, CM-Build, NaN, CM-Managed, JOCs, ..., DDC Managed/Consultant, Pass Through, DDC Managed Consultant, DEP Managed, None]
 
 'MultipleFMSIds': [FALSE, TRUE]

 'DesignFiscalYear'
 
 'ConstructionFiscalYear'

In [46]:
df_SP.ConstructionContractType.unique()

[DDC Managed, CM-Build, NaN, CM-Managed, JOCs, ..., DDC Managed/Consultant, Pass Through, DDC Managed Consultant, DEP Managed, None]
Length: 15
Categories (14, object): [DDC Managed, CM-Build, CM-Managed, JOCs, ..., Pass Through, DDC Managed Consultant, DEP Managed, None]

** For possible features replace na with "MissingValue", for the purpose of exploration and prediction **

In [47]:
fill_NA_list = ['DivisionName','UnitName','Borough','Sponsor','PhaseName','ProjectType','Priority',
 'DesignContractType','ConstructionContractType','SourceSystem','MultipleFMSIds','DesignFiscalYear',
 'ConstructionFiscalYear']

for col in fill_NA_list:
    if str(df_SP[col].dtypes) == 'category':
        df_SP[col] = df_SP[col].cat.add_categories("MissingValue").fillna("MissingValue")
    else:
        df_SP[col] = df_SP[col].fillna("MissingValue")

In [48]:
field_details(df_SP)

ProjectId: max count = 1, null values = 0, category values = 857 category values

ActProjectStart: max count = 54, null values = 0, category values = 523 category values

ActDesignStart: max count = 131, null values = 0, category values = 516 category values

ActDesignCompletion: max count = 408, null values = 0, category values = 339 category values

ActConstructionStart: max count = 438, null values = 0, category values = 316 category values

ActConstructionCompletion: max count = 613, null values = 0, category values = 207 category values

ActSchematicDesignCompletion: max count = 392, null values = 0, category values = 361 category values

ActDesignDevelopmentCompletion: max count = 713, null values = 0, category values = 123 category values

ActProjectCloseout: max count = 838, null values = 0, category values = [numpy.datetime64('2017-02-07T00:00:00.000000000'), numpy.datetime64('1900-01-01T00:00:00.000000000'), numpy.datetime64('2016-11-14T00:00:00.000000000'), numpy.datetime64(

# Output the cleaned file to a dataframe

In [49]:
#output df_SP to a file
df_SP.to_pickle('DDC_data_cleaned.pkl')